In [21]:
import os
import json
# from fastapi import FastAPI
# from pydantic import BaseModel
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, OpenAI
# from langchain_openai import ChatOpenAI
# from langchain.prompts import PromptTemplate
# from langchain.memory import ConversationBufferMemory
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.schema import Document
from langchain_community.document_loaders import TextLoader
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA

In [22]:

# Load environment variables
load_dotenv()
open_api_key = os.getenv("OPENAI_API_KEY")
if not open_api_key:
    raise ValueError("OPENAI_API_KEY is not set")

In [23]:
loader = TextLoader("data/data.txt", encoding="utf-8")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    length_function=len
)



In [24]:
docs = text_splitter.split_documents(documents)
docs

[Document(metadata={'source': 'data/data.txt'}, page_content='Azercell is the only company in Azerbaijan and CIS region which has been awarded Platinum'),
 Document(metadata={'source': 'data/data.txt'}, page_content='Certificate of International “Investors in People” Standard. The mobile operator is the only'),
 Document(metadata={'source': 'data/data.txt'}, page_content='company in the country to receive Gold Award in nominations of "Company of the Year" and “The Most'),
 Document(metadata={'source': 'data/data.txt'}, page_content='Innovative Company of the Year” from the International Business Award  STEVIE.')]

In [25]:
embeddings = OpenAIEmbeddings()
faiss_search = FAISS.from_documents(docs, embeddings)

In [26]:
query1 = "What unique award has Azercell won?"
query1_answer = faiss_search.similarity_search(query1)

In [27]:
query1_answer
#query1_answer[0].page_content

[Document(id='dd0a31e6-ea61-4815-8ca8-08d0da4378aa', metadata={'source': 'data/data.txt'}, page_content='Azercell is the only company in Azerbaijan and CIS region which has been awarded Platinum'),
 Document(id='d04169ee-e7f3-4fd0-99b4-b9fa20f37d3a', metadata={'source': 'data/data.txt'}, page_content='Innovative Company of the Year” from the International Business Award  STEVIE.'),
 Document(id='5a866810-e27b-4c4a-8f8a-9d10330afbe2', metadata={'source': 'data/data.txt'}, page_content='Certificate of International “Investors in People” Standard. The mobile operator is the only'),
 Document(id='47b30eaf-8849-48c8-a650-47eaf7f90df2', metadata={'source': 'data/data.txt'}, page_content='company in the country to receive Gold Award in nominations of "Company of the Year" and “The Most')]

In [28]:
docs_and_scores = faiss_search.similarity_search_with_score(query1)
docs_and_scores

[(Document(id='dd0a31e6-ea61-4815-8ca8-08d0da4378aa', metadata={'source': 'data/data.txt'}, page_content='Azercell is the only company in Azerbaijan and CIS region which has been awarded Platinum'),
  0.2318678),
 (Document(id='d04169ee-e7f3-4fd0-99b4-b9fa20f37d3a', metadata={'source': 'data/data.txt'}, page_content='Innovative Company of the Year” from the International Business Award  STEVIE.'),
  0.38795558),
 (Document(id='5a866810-e27b-4c4a-8f8a-9d10330afbe2', metadata={'source': 'data/data.txt'}, page_content='Certificate of International “Investors in People” Standard. The mobile operator is the only'),
  0.43471655),
 (Document(id='47b30eaf-8849-48c8-a650-47eaf7f90df2', metadata={'source': 'data/data.txt'}, page_content='company in the country to receive Gold Award in nominations of "Company of the Year" and “The Most'),
  0.43757594)]

In [29]:
retreiver = faiss_search.as_retriever()
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retreiver)

In [30]:
retreiver_query = "What unique award has Azercell won?"
results = qa.invoke(retreiver_query)

In [31]:
results

{'query': 'What unique award has Azercell won?',
 'result': '\nAzercell has won the Platinum Innovative Company of the Year award from the International Business Award STEVIE.'}